<h1 align=center><font size = 5>Segmenting and Clustering Neighborhoods in Toronto</font></h1>


### Question 1: Extracting the data from the Wikipedia page

## Introduction

In this assignment, we will be required to segment and cluster the neighborhoods in the city of Toronto. The data exists on the Wikipedia page so the first step would be to scrape the data from the Wikipedia page.


## 1. Scraping of data
Import the dependencies:

In [3]:
import pandas as pd
from bs4 import BeautifulSoup
import requests
import numpy as np
from geopy.geocoders import Nominatim 
from pandas.io.json import json_normalize

!pip install folium
import folium

from sklearn.cluster import KMeans
#Matplotlib and other plotting modules
import matplotlib.cm as cm
import matplotlib.colors as colors
print('Dependencies imported!')

     |████████████████████████████████| 93 kB 3.2 MB/s  eta 0:00:01
Dependencies imported!


Extract the table containing data:

In [4]:
Data = requests.get("https://en.wikipedia.org/wiki/List_of_postal_codes_of_Canada:_M").text
soup = BeautifulSoup(Data, 'lxml')

table = soup.find("table")
table_rows = table.tbody.find_all("tr")

res = []
for tr in table_rows:
    td = tr.find_all("td")
    row = [tr.text for tr in td]
    
    if row != [] and row[1] != "Not assigned":
        if "Not assigned" in row[2]: 
            row[2] = row[1]
        res.append(row)

# Dataframe
df = pd.DataFrame(res, columns = ["PostalCode", "Borough", "Neighborhood"])
df.head()

,PostalCode,Borough,Neighborhood
0,M1A\n,Not assigned\n,Not assigned\n
1,M2A\n,Not assigned\n,Not assigned\n
2,M3A\n,North York\n,Parkwoods\n
3,M4A\n,North York\n,Victoria Village\n
4,M5A\n,Downtown Toronto\n,"Regent Park, Harbourfront\n"


Removing \n from all entries:

In [5]:
df['PostalCode'] = df['PostalCode'].str.replace("\n","")
df['Borough'] = df['Borough'].str.replace("\n","")
df['Neighborhood'] = df['Neighborhood'].str.replace("\n","")
df.head()

,PostalCode,Borough,Neighborhood
0,M1A,Not assigned,Not assigned
1,M2A,Not assigned,Not assigned
2,M3A,North York,Parkwoods
3,M4A,North York,Victoria Village
4,M5A,Downtown Toronto,"Regent Park, Harbourfront"


Only process the cells that have an assigned borough. Ignore cells with a borough that is Not assigned:

In [6]:
df = df[~df['Borough'].isin(['Not assigned'])]
df.head()

,PostalCode,Borough,Neighborhood
2,M3A,North York,Parkwoods
3,M4A,North York,Victoria Village
4,M5A,Downtown Toronto,"Regent Park, Harbourfront"
5,M6A,North York,"Lawrence Manor, Lawrence Heights"
6,M7A,Downtown Toronto,"Queen's Park, Ontario Provincial Government"


Group neighborhoods with the same postal code:

In [7]:
df = df.groupby(["PostalCode", "Borough"])["Neighborhood"].apply(", ".join).reset_index()
df.head()

,PostalCode,Borough,Neighborhood
0,M1B,Scarborough,"Malvern, Rouge"
1,M1C,Scarborough,"Rouge Hill, Port Union, Highland Creek"
2,M1E,Scarborough,"Guildwood, Morningside, West Hill"
3,M1G,Scarborough,Woburn
4,M1H,Scarborough,Cedarbrae


Get shape of the dataframe:

In [8]:
print('Shape of the dataframe:', df.shape)

Shape of the dataframe: (103, 3)


### Question 2: Get the latitude and the longitude coordinates of each neighborhood

In [9]:
def get_geocode(postal_code):
    lat_lng_coords = None # Start with None
    while(lat_lng_coords is None):
        g = geocoder.google('{}, Toronto, Ontario'.format(postal_code))
        lat_lng_coords = g.latlng
    latitude = lat_lng_coords[0]
    longitude = lat_lng_coords[1]
    return latitude,longitude

In [16]:
df_geo=pd.read_csv('http://cocl.us/Geospatial_data')
df_geo.head()

,Postal Code,Latitude,Longitude
0,M1B,43.806686,-79.194353
1,M1C,43.784535,-79.160497
2,M1E,43.763573,-79.188711
3,M1G,43.770992,-79.216917
4,M1H,43.773136,-79.239476


Merge two dataframes: df_geo and df:

In [17]:
df_toronto = pd.merge(df, df_geo, how='left', left_on = 'PostalCode', right_on = 'Postal Code')
df_toronto.head()

,PostalCode,Borough,Neighborhood,Postal Code,Latitude,Longitude
0,M1B,Scarborough,"Malvern, Rouge",M1B,43.806686,-79.194353
1,M1C,Scarborough,"Rouge Hill, Port Union, Highland Creek",M1C,43.784535,-79.160497
2,M1E,Scarborough,"Guildwood, Morningside, West Hill",M1E,43.763573,-79.188711
3,M1G,Scarborough,Woburn,M1G,43.770992,-79.216917
4,M1H,Scarborough,Cedarbrae,M1H,43.773136,-79.239476


Drop 'Postal Code' column:

In [18]:
df_toronto.drop('Postal Code', axis=1, inplace=True)
df_toronto.head()

,PostalCode,Borough,Neighborhood,Latitude,Longitude
0,M1B,Scarborough,"Malvern, Rouge",43.806686,-79.194353
1,M1C,Scarborough,"Rouge Hill, Port Union, Highland Creek",43.784535,-79.160497
2,M1E,Scarborough,"Guildwood, Morningside, West Hill",43.763573,-79.188711
3,M1G,Scarborough,Woburn,43.770992,-79.216917
4,M1H,Scarborough,Cedarbrae,43.773136,-79.239476
